# Phase B: Sector ETF Short-term Momentum

本ノートブックは、Short-term Momentum 論文の実装枠組みを **9セクターSPDR ETF** に置き換えて再現する。

- ユニバース: `XLB,XLE,XLF,XLI,XLK,XLP,XLU,XLV,XLY`
- 形成シグナル: 前月リターン + 前月出来高ベースの `Volume z-score`
- ソート: 3x3 条件付きソート（Return tercile -> Turnover tercile）
- 時点整合: 形成月 `m` でシグナル生成し、保有月 `m+1` に適用
- コスト: one-way turnover × 10bps


## 1. Setup


In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from datetime import datetime
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
plt.style.use('ggplot')

TICKERS = ['XLB','XLE','XLF','XLI','XLK','XLP','XLU','XLV','XLY']
START_DATE = pd.Timestamp('2005-01-01')
COST_BPS = 10
OUT_ROOT = Path('outputs/phase_b')
FIG_DIR = OUT_ROOT / 'figures'
TAB_DIR = OUT_ROOT / 'tables'
DATA_PATH = Path('data/prices.csv')

for d in [OUT_ROOT, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print('tickers:', TICKERS)
print('start_date:', START_DATE.date())
print('cost_bps:', COST_BPS)
print('data_path:', DATA_PATH.resolve())


## 2. Data Contract Check


In [ ]:
def read_yfinance_multindex_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        raise FileNotFoundError(f'Missing required input: {path}')

    with path.open('r', encoding='utf-8') as f:
        l1 = f.readline().strip().split(',')
        l2 = f.readline().strip().split(',')

    # yfinance exports often include Price/Ticker header rows plus one extra row.
    if l1 and l1[0] == 'Price' and l2 and l2[0] == 'Ticker':
        df = pd.read_csv(path, header=[0, 1], skiprows=[2], index_col=0)
    else:
        df = pd.read_csv(path, header=[0, 1], index_col=0)

    try:
        df.index = pd.to_datetime(df.index)
    except Exception as e:
        raise ValueError(f'Index must be parseable datetime in {path}: {e}')

    if not isinstance(df.columns, pd.MultiIndex):
        raise ValueError('prices.csv must have MultiIndex columns: (field, ticker)')

    lvl0 = set(df.columns.get_level_values(0))
    for req in ['Adj Close', 'Volume']:
        if req not in lvl0:
            raise ValueError(f"Missing field '{req}' in prices.csv columns")

    adj_cols = set(df['Adj Close'].columns)
    vol_cols = set(df['Volume'].columns)

    missing_adj = [t for t in TICKERS if t not in adj_cols]
    missing_vol = [t for t in TICKERS if t not in vol_cols]

    if missing_adj or missing_vol:
        raise ValueError(
            'Data contract violation\n'
            f'  missing Adj Close tickers: {missing_adj}\n'
            f'  missing Volume tickers: {missing_vol}'
        )

    return df


raw = read_yfinance_multindex_csv(DATA_PATH)
print('contract check passed')
print('date range:', raw.index.min(), '->', raw.index.max())
print('rows:', len(raw))


## 3. Data Load


In [ ]:
adj_close = raw['Adj Close'][TICKERS].copy()
volume = raw['Volume'][TICKERS].copy()

adj_close = adj_close.sort_index()
volume = volume.sort_index()

adj_close = adj_close[~adj_close.index.duplicated(keep='last')]
volume = volume[~volume.index.duplicated(keep='last')]

adj_close = adj_close.loc[adj_close.index >= START_DATE]
volume = volume.loc[volume.index >= START_DATE]

adj_close = adj_close.astype(float).ffill()
volume = volume.astype(float)

if not adj_close.index.equals(volume.index):
    idx = adj_close.index.intersection(volume.index)
    adj_close = adj_close.loc[idx]
    volume = volume.loc[idx]

if adj_close.empty:
    raise ValueError('No data after START_DATE filter')

print('loaded daily rows:', len(adj_close))
print('adj_close missing total:', int(adj_close.isna().sum().sum()))
print('volume missing total:', int(volume.isna().sum().sum()))


## 4. Signal Engineering


In [ ]:
def rolling_zscore(df: pd.DataFrame, window: int = 12, min_periods: int = 12) -> pd.DataFrame:
    mu = df.rolling(window=window, min_periods=min_periods).mean()
    sd = df.rolling(window=window, min_periods=min_periods).std(ddof=0)
    return (df - mu) / sd


def monthly_from_daily(adj_close_daily: pd.DataFrame, volume_daily: pd.DataFrame):
    monthly_price = adj_close_daily.resample('ME').last()
    monthly_ret = monthly_price.pct_change()

    monthly_vol_sum = volume_daily.resample('ME').sum(min_count=1)
    turnover_base = np.log(monthly_vol_sum.replace(0, np.nan))
    turnover_proxy = rolling_zscore(turnover_base, window=12, min_periods=12)

    return monthly_price, monthly_ret, monthly_vol_sum, turnover_proxy


def monthly_excluding_last_3_days(adj_close_daily: pd.DataFrame, volume_daily: pd.DataFrame):
    idx = adj_close_daily.index
    month_key = idx.to_period('M')

    rank_from_end = idx.to_series(index=idx).groupby(month_key).cumcount(ascending=False) + 1
    keep_mask = rank_from_end > 3

    # Keep only rows excluding each month's last 3 trading days.
    adj_keep = adj_close_daily.loc[keep_mask.values].copy()
    vol_keep = volume_daily.loc[keep_mask.values].copy()

    month_key_keep = adj_keep.index.to_period('M')
    price_eom = adj_keep.groupby(month_key_keep).last()
    vol_eom = vol_keep.groupby(month_key_keep).sum(min_count=1)

    price_eom.index = price_eom.index.to_timestamp(how='end').normalize()
    vol_eom.index = vol_eom.index.to_timestamp(how='end').normalize()

    ret_eom = price_eom.pct_change()
    turnover_eom = rolling_zscore(np.log(vol_eom.replace(0, np.nan)), window=12, min_periods=12)

    return price_eom, ret_eom, vol_eom, turnover_eom


monthly_price, monthly_ret, monthly_vol, turnover_proxy = monthly_from_daily(adj_close, volume)
monthly_price_eom, monthly_ret_eom, monthly_vol_eom, turnover_proxy_eom = monthly_excluding_last_3_days(adj_close, volume)

print('monthly rows:', len(monthly_ret))
print('monthly rows (EOM excl.):', len(monthly_ret_eom))


## 5. Portfolio Construction


In [ ]:
@dataclass
class StrategyResult:
    name: str
    ret_gross: pd.Series
    turnover: pd.Series
    ret_net: pd.Series
    weights: pd.DataFrame
    meta: pd.DataFrame


def safe_tercile_labels(series: pd.Series) -> pd.Series:
    if series.notna().sum() < 3:
        return pd.Series(index=series.index, dtype='float64')
    ranked = series.rank(method='first')
    try:
        return pd.qcut(ranked, 3, labels=[0, 1, 2])
    except Exception:
        # fallback: deterministic split by rank
        n = len(ranked)
        bins = np.ceil((ranked / n) * 3).clip(1, 3) - 1
        return bins.astype(int)


def build_conditional_3x3_strategy(
    monthly_ret: pd.DataFrame,
    turnover_signal: pd.DataFrame,
    strategy_name: str,
    turnover_bucket: int,
    cost_bps: float = 10.0,
) -> StrategyResult:
    months = monthly_ret.index
    assets = list(monthly_ret.columns)

    w_list = []
    gross = []
    one_way_turn = []
    meta_rows = []
    prev_w = pd.Series(0.0, index=assets)

    # formation month m, holding month m+1
    for i in range(len(months) - 1):
        m = months[i]
        h = months[i + 1]

        r_form = monthly_ret.loc[m]
        t_form = turnover_signal.loc[m]
        r_hold = monthly_ret.loc[h]

        valid = r_form.notna() & t_form.notna() & r_hold.notna()
        valid_assets = r_form.index[valid]

        if len(valid_assets) < 6:
            continue

        # first split by return terciles
        ret_grp = safe_tercile_labels(r_form.loc[valid_assets])

        # conditional turnover split within each return group
        cell = {}
        for rg in [0, 1, 2]:
            bucket_assets = ret_grp.index[ret_grp.astype(float) == rg]
            if len(bucket_assets) == 0:
                continue
            t_grp = safe_tercile_labels(t_form.loc[bucket_assets])
            for tg in [0, 1, 2]:
                cell[(rg, tg)] = list(t_grp.index[t_grp.astype(float) == tg])

        req_w = cell.get((2, turnover_bucket), [])
        req_l = cell.get((0, turnover_bucket), [])

        # acceptance: required cells must not be empty when month is used
        if len(req_w) == 0 or len(req_l) == 0:
            continue

        w = pd.Series(0.0, index=assets)
        w.loc[req_w] = 0.5 / len(req_w)
        w.loc[req_l] = -0.5 / len(req_l)

        # leak guard: formed at m, applied to h
        assert h > m, f'holding month must be after formation month: {m} -> {h}'

        g = float((w * r_hold).sum())
        t = float(0.5 * (w - prev_w).abs().sum())
        n = g - t * (cost_bps / 10000.0)

        w_list.append(w.rename(h))
        gross.append((h, g))
        one_way_turn.append((h, t))
        meta_rows.append({
            'formation_month': m,
            'holding_month': h,
            'n_valid_assets': int(len(valid_assets)),
            'n_winners': int(len(req_w)),
            'n_losers': int(len(req_l)),
            'turnover_bucket': int(turnover_bucket),
        })
        prev_w = w

    if not w_list:
        raise ValueError(f'No valid monthly portfolios for {strategy_name}. Check signals/data completeness.')

    W = pd.DataFrame(w_list)
    ret_gross = pd.Series(dict(gross), name=f'{strategy_name}_gross').sort_index()
    turn = pd.Series(dict(one_way_turn), name=f'{strategy_name}_turnover').sort_index()
    ret_net = (ret_gross - turn * (cost_bps / 10000.0)).rename(f'{strategy_name}_net')

    # acceptance: gross-net consistency
    diff = (ret_gross - ret_net - turn * (cost_bps / 10000.0)).abs().max()
    assert diff < 1e-12, f'cost consistency failed for {strategy_name}: {diff}'

    meta = pd.DataFrame(meta_rows).set_index('holding_month').sort_index()
    return StrategyResult(strategy_name, ret_gross, turn, ret_net, W.sort_index(), meta)


def build_simple_2x2_strategy(
    monthly_ret: pd.DataFrame,
    turnover_signal: pd.DataFrame,
    strategy_name: str,
    turnover_high: bool = True,
    cost_bps: float = 10.0,
) -> StrategyResult:
    months = monthly_ret.index
    assets = list(monthly_ret.columns)

    w_list = []
    gross = []
    one_way_turn = []
    meta_rows = []
    prev_w = pd.Series(0.0, index=assets)

    for i in range(len(months) - 1):
        m = months[i]
        h = months[i + 1]

        r_form = monthly_ret.loc[m]
        t_form = turnover_signal.loc[m]
        r_hold = monthly_ret.loc[h]

        valid = r_form.notna() & t_form.notna() & r_hold.notna()
        if valid.sum() < 4:
            continue

        rv = r_form[valid]
        tv = t_form[valid]

        r_med = rv.median()
        t_med = tv.median()

        winners = rv.index[rv >= r_med]
        losers = rv.index[rv < r_med]

        if turnover_high:
            high_turn = tv.index[tv >= t_med]
            w_assets = list(set(winners).intersection(high_turn))
            l_assets = list(set(losers).intersection(high_turn))
        else:
            low_turn = tv.index[tv < t_med]
            w_assets = list(set(winners).intersection(low_turn))
            l_assets = list(set(losers).intersection(low_turn))

        if len(w_assets) == 0 or len(l_assets) == 0:
            continue

        w = pd.Series(0.0, index=assets)
        w.loc[w_assets] = 0.5 / len(w_assets)
        w.loc[l_assets] = -0.5 / len(l_assets)

        assert h > m, f'holding month must be after formation month: {m} -> {h}'

        g = float((w * r_hold).sum())
        t = float(0.5 * (w - prev_w).abs().sum())
        n = g - t * (cost_bps / 10000.0)

        w_list.append(w.rename(h))
        gross.append((h, g))
        one_way_turn.append((h, t))
        meta_rows.append({
            'formation_month': m,
            'holding_month': h,
            'n_valid_assets': int(valid.sum()),
            'n_winners': int(len(w_assets)),
            'n_losers': int(len(l_assets)),
            'turnover_high': bool(turnover_high),
        })
        prev_w = w

    if not w_list:
        raise ValueError(f'No valid monthly portfolios for {strategy_name}')

    W = pd.DataFrame(w_list)
    ret_gross = pd.Series(dict(gross), name=f'{strategy_name}_gross').sort_index()
    turn = pd.Series(dict(one_way_turn), name=f'{strategy_name}_turnover').sort_index()
    ret_net = (ret_gross - turn * (cost_bps / 10000.0)).rename(f'{strategy_name}_net')

    diff = (ret_gross - ret_net - turn * (cost_bps / 10000.0)).abs().max()
    assert diff < 1e-12, f'cost consistency failed for {strategy_name}: {diff}'

    meta = pd.DataFrame(meta_rows).set_index('holding_month').sort_index()
    return StrategyResult(strategy_name, ret_gross, turn, ret_net, W.sort_index(), meta)


baseline_stmom = build_conditional_3x3_strategy(
    monthly_ret=monthly_ret,
    turnover_signal=turnover_proxy,
    strategy_name='STMOM_ETF',
    turnover_bucket=2,
    cost_bps=COST_BPS,
)
baseline_strev = build_conditional_3x3_strategy(
    monthly_ret=monthly_ret,
    turnover_signal=turnover_proxy,
    strategy_name='STREV_ETF',
    turnover_bucket=0,
    cost_bps=COST_BPS,
)

eom_stmom = build_conditional_3x3_strategy(
    monthly_ret=monthly_ret_eom,
    turnover_signal=turnover_proxy_eom,
    strategy_name='STMOM_ETF_EOMEXCL',
    turnover_bucket=2,
    cost_bps=COST_BPS,
)

eom_strev = build_conditional_3x3_strategy(
    monthly_ret=monthly_ret_eom,
    turnover_signal=turnover_proxy_eom,
    strategy_name='STREV_ETF_EOMEXCL',
    turnover_bucket=0,
    cost_bps=COST_BPS,
)

simple_2x2 = build_simple_2x2_strategy(
    monthly_ret=monthly_ret,
    turnover_signal=turnover_proxy,
    strategy_name='STMOM_2x2_HIGH',
    turnover_high=True,
    cost_bps=COST_BPS,
)

print('baseline STMOM months:', len(baseline_stmom.ret_gross))
print('baseline STREV months:', len(baseline_strev.ret_gross))
print('eom STMOM months:', len(eom_stmom.ret_gross))
print('2x2 months:', len(simple_2x2.ret_gross))


## 6. Backtest


In [ ]:
def annualized_metrics(ret: pd.Series, periods_per_year: int = 12) -> dict:
    ret = ret.dropna()
    if len(ret) == 0:
        return {'n_months': 0, 'CAGR': np.nan, 'Vol': np.nan, 'Sharpe': np.nan, 'MaxDD': np.nan, 'HitRate': np.nan}

    equity = (1 + ret).cumprod()
    years = len(ret) / periods_per_year
    cagr = equity.iloc[-1] ** (1 / years) - 1 if years > 0 else np.nan
    vol = ret.std(ddof=1) * np.sqrt(periods_per_year)
    sharpe = (ret.mean() * periods_per_year) / vol if vol and vol > 0 else np.nan
    dd = equity / equity.cummax() - 1
    maxdd = dd.min()
    hit = (ret > 0).mean()

    return {
        'n_months': int(len(ret)),
        'CAGR': float(cagr),
        'Vol': float(vol),
        'Sharpe': float(sharpe),
        'MaxDD': float(maxdd),
        'HitRate': float(hit),
    }


strategy_returns = pd.concat([
    baseline_stmom.ret_gross,
    baseline_stmom.turnover,
    baseline_stmom.ret_net,
    baseline_strev.ret_gross,
    baseline_strev.ret_net,
    eom_stmom.ret_gross,
    eom_stmom.ret_net,
    eom_strev.ret_gross,
    eom_strev.ret_net,
    simple_2x2.ret_gross,
    simple_2x2.ret_net,
], axis=1).sort_index()

metrics_rows = []
for c in strategy_returns.columns:
    if c.endswith('_gross') or c.endswith('_net'):
        m = annualized_metrics(strategy_returns[c])
        m['series'] = c
        metrics_rows.append(m)

metrics_summary = pd.DataFrame(metrics_rows).set_index('series').sort_index()
metrics_summary


## 7. Robustness


In [ ]:
def build_signal_for_lookback(monthly_price: pd.DataFrame, lookback: int) -> pd.DataFrame:
    return monthly_price.pct_change(lookback)


def fama_macbeth_monthly(
    next_ret: pd.DataFrame,
    r1_0: pd.DataFrame,
    turnover: pd.DataFrame,
) -> pd.DataFrame:
    rows = []
    months = next_ret.index.intersection(r1_0.index).intersection(turnover.index)

    for m in months:
        y = next_ret.loc[m]
        x1 = r1_0.loc[m]
        x2 = turnover.loc[m]

        df = pd.concat([y.rename('y'), x1.rename('r1_0'), x2.rename('turn')], axis=1).dropna()
        if len(df) < 5:
            continue

        X = np.column_stack([
            np.ones(len(df)),
            df['r1_0'].values,
            df['turn'].values,
            (df['r1_0'] * df['turn']).values,
        ])
        beta = np.linalg.lstsq(X, df['y'].values, rcond=None)[0]
        rows.append({
            'month': m,
            'alpha': beta[0],
            'beta_r1_0': beta[1],
            'beta_turn': beta[2],
            'beta_interaction': beta[3],
            'n_assets': int(len(df)),
        })

    coef_ts = pd.DataFrame(rows).set_index('month').sort_index()
    if coef_ts.empty:
        raise ValueError('Fama-MacBeth: no valid monthly regressions')

    summary = []
    for col in ['alpha', 'beta_r1_0', 'beta_turn', 'beta_interaction']:
        s = coef_ts[col].dropna()
        mean = s.mean()
        std = s.std(ddof=1)
        t = mean / (std / np.sqrt(len(s))) if std > 0 else np.nan
        summary.append({'coef': col, 'mean': mean, 't_stat': t, 'n_months': int(len(s))})

    return coef_ts, pd.DataFrame(summary).set_index('coef')


# 形成窓感度: 1M / 3M / 6M
lookback_results = {}
for lb in [1, 3, 6]:
    sig = build_signal_for_lookback(monthly_price, lb)
    res = build_conditional_3x3_strategy(
        monthly_ret=sig,
        turnover_signal=turnover_proxy,
        strategy_name=f'STMOM_ETF_LB{lb}',
        turnover_bucket=2,
        cost_bps=COST_BPS,
    )
    lookback_results[lb] = res

# EOM除外有無の差分統計
aligned = pd.concat([
    baseline_stmom.ret_gross.rename('base'),
    eom_stmom.ret_gross.rename('eom'),
], axis=1).dropna()

diff = aligned['eom'] - aligned['base']
mean_diff = diff.mean()
std_diff = diff.std(ddof=1)
t_diff = mean_diff / (std_diff / np.sqrt(len(diff))) if std_diff > 0 else np.nan

# Fama-MacBeth: y_{t+1} regression on month-t signals
next_ret = monthly_ret.shift(-1)
fm_ts, fm_summary = fama_macbeth_monthly(next_ret=next_ret, r1_0=monthly_ret, turnover=turnover_proxy)

# サブ期間比較
subperiods = {
    '2005-2012': ('2005-01-01', '2012-12-31'),
    '2013-2019': ('2013-01-01', '2019-12-31'),
    '2020-latest': ('2020-01-01', None),
}

sub_rows = []
for name, (s, e) in subperiods.items():
    s = pd.Timestamp(s)
    e = pd.Timestamp(e) if e else strategy_returns.index.max()
    window = strategy_returns.loc[(strategy_returns.index >= s) & (strategy_returns.index <= e)]

    for col in ['STMOM_ETF_gross', 'STMOM_ETF_net', 'STMOM_ETF_EOMEXCL_gross', 'STMOM_ETF_EOMEXCL_net']:
        if col not in window.columns:
            continue
        met = annualized_metrics(window[col])
        met['subperiod'] = name
        met['series'] = col
        sub_rows.append(met)

subperiod_summary = pd.DataFrame(sub_rows)

robustness_summary = pd.DataFrame([
    {
        'test': 'EOM_exclusion_minus_baseline_mean',
        'value': float(mean_diff),
        't_stat': float(t_diff),
        'n_months': int(len(diff)),
    },
    {
        'test': 'lookback_1M_sharpe',
        'value': float(annualized_metrics(lookback_results[1].ret_gross)['Sharpe']),
        't_stat': np.nan,
        'n_months': int(annualized_metrics(lookback_results[1].ret_gross)['n_months']),
    },
    {
        'test': 'lookback_3M_sharpe',
        'value': float(annualized_metrics(lookback_results[3].ret_gross)['Sharpe']),
        't_stat': np.nan,
        'n_months': int(annualized_metrics(lookback_results[3].ret_gross)['n_months']),
    },
    {
        'test': 'lookback_6M_sharpe',
        'value': float(annualized_metrics(lookback_results[6].ret_gross)['Sharpe']),
        't_stat': np.nan,
        'n_months': int(annualized_metrics(lookback_results[6].ret_gross)['n_months']),
    },
])

print('robustness summary:')
robustness_summary


## 8. Diagnostics


In [ ]:
# ソート安定性テスト（required cells not empty for used months）
assert (baseline_stmom.meta['n_winners'] > 0).all() and (baseline_stmom.meta['n_losers'] > 0).all()
assert (baseline_strev.meta['n_winners'] > 0).all() and (baseline_strev.meta['n_losers'] > 0).all()

# リーク防止テスト（形成月m -> 翌月m+1）
lag_ok = (baseline_stmom.meta.index.to_period('M') == (baseline_stmom.meta['formation_month'].dt.to_period('M') + 1)).all()
assert bool(lag_ok), 'Leak guard failed: holding month is not formation_month + 1M'

# シグナル診断
signal_diag = pd.DataFrame({
    'ret_missing_ratio': monthly_ret.isna().mean(),
    'turn_missing_ratio': turnover_proxy.isna().mean(),
    'ret_mean': monthly_ret.mean(),
    'ret_std': monthly_ret.std(ddof=1),
    'turn_mean': turnover_proxy.mean(),
    'turn_std': turnover_proxy.std(ddof=1),
}).sort_index()

corr_diag = pd.DataFrame({
    'ret_turn_corr': [
        pd.concat([monthly_ret[t], turnover_proxy[t]], axis=1).corr().iloc[0, 1]
        for t in TICKERS
    ]
}, index=TICKERS)

signal_diagnostics = signal_diag.join(corr_diag)
signal_diagnostics


## 9. Export


In [ ]:
# Tables
strategy_returns.to_csv(TAB_DIR / 'strategy_returns_monthly.csv', index=True)
metrics_summary.to_csv(TAB_DIR / 'metrics_summary.csv', index=True)
robustness_summary.to_csv(TAB_DIR / 'robustness_summary.csv', index=False)
signal_diagnostics.to_csv(TAB_DIR / 'signal_diagnostics.csv', index=True)
fm_summary.to_csv(TAB_DIR / 'fama_macbeth_summary.csv', index=True)
subperiod_summary.to_csv(TAB_DIR / 'subperiod_summary.csv', index=False)

# Figures
def plot_equity_curve(ret_series: pd.Series, title: str, out_path: Path):
    eq = (1 + ret_series.dropna()).cumprod()
    fig, ax = plt.subplots(figsize=(10, 4))
    eq.plot(ax=ax, lw=1.4)
    ax.set_title(title)
    ax.set_ylabel('Cumulative Growth')
    ax.grid(True, alpha=0.3)
    fig.tight_layout()
    fig.savefig(out_path, dpi=150)
    plt.close(fig)


plot_equity_curve(strategy_returns['STMOM_ETF_gross'], 'STMOM ETF Gross Equity Curve', FIG_DIR / 'stmom_etf_gross_equity.png')
plot_equity_curve(strategy_returns['STMOM_ETF_net'], 'STMOM ETF Net Equity Curve', FIG_DIR / 'stmom_etf_net_equity.png')
plot_equity_curve(strategy_returns['STMOM_ETF_EOMEXCL_gross'], 'STMOM ETF EOM Excl Gross Equity Curve', FIG_DIR / 'stmom_etf_eom_gross_equity.png')

fig, ax = plt.subplots(figsize=(8, 4))
(strategy_returns['STMOM_ETF_EOMEXCL_gross'] - strategy_returns['STMOM_ETF_gross']).dropna().hist(bins=24, ax=ax)
ax.set_title('EOM Exclusion - Baseline Monthly Return Difference')
ax.grid(True, alpha=0.3)
fig.tight_layout()
fig.savefig(FIG_DIR / 'eom_minus_baseline_hist.png', dpi=150)
plt.close(fig)

saved_files = sorted([str(p) for p in OUT_ROOT.rglob('*') if p.is_file()])
print('Saved files:')
for p in saved_files:
    print('-', p)


## 10. Run Log


In [ ]:
run_ts = datetime.now().strftime('%Y%m%d_%H%M%S')
runlog_path = OUT_ROOT / f'phaseB_runlog_{run_ts}.md'

lines = []
lines.append('# Phase B Run Log (Sector ETF)')
lines.append('')
lines.append(f'- executed_at: {datetime.now().isoformat(timespec="seconds")}')
lines.append(f'- notebook: {Path("short_term_momentum_sector_etf.ipynb").resolve()}')
lines.append(f'- data_source: {DATA_PATH.resolve()}')
lines.append(f'- universe: {", ".join(TICKERS)}')
lines.append(f'- sample_start: {START_DATE.date()}')
lines.append(f'- cost_bps: {COST_BPS}')
lines.append('')
lines.append('## Outputs')
for p in sorted([str(p) for p in OUT_ROOT.rglob('*') if p.is_file()]):
    lines.append(f'- {p}')

runlog_path.write_text('\n'.join(lines), encoding='utf-8')
print('Run log written:', runlog_path)
print(runlog_path.read_text(encoding='utf-8'))
